In [ ]:
def forward(self, x, temb):
    """
    x: latent feature map          (B, C, H, W)
       - VAE encoder가 만든 latent에서 denoising을 수행하는 feature
    temb: timestep embedding        (B, temb_dim)
       - diffusion step t → sinusoidal embedding → dense → nonlinearity → dense
       - U-Net 전체에서 noise level/time 정보를 전달하기 위한 vector

    이 블록은 U-Net의 한 레이어에서 residual path 연산을 수행합니다.
    LDM U-Net에서는 이 블록이 down/up sampling, mid layer 등에 반복적으로 쓰입니다. :contentReference[oaicite:2]{index=2}
    """

    # ================= Residual Branch 시작 =================
    h = x  # 입력 복사 (residual 합을 위한 기본 분기)

    # ----- 1. 정규화(norm) → 활성화(nonlinearity) → conv1 -----
    h = self.norm1(h)
    h = nonlinearity(h)        # 보통 SiLU(x) = x * sigmoid(x)
    h = self.conv1(h)          # latent 공간에서의 convolution

    # ----- 2. Time embedding 주입 (noise scale conditioning) -----
    if temb is not None:
        # temb: (B, temb_dim) → nonlinearity → linear projection → (B, C)
        # → [:, :, None, None]로 reshape하여 spatial 전체에 broadcast
        # Latent space 전체를 noise level 조건으로 강화
        h = h + self.temb_proj(nonlinearity(temb))[:, :, None, None]

    # ----- 3. 정규화(norm) → 활성화 → dropout → conv2 -----
    h = self.norm2(h)
    h = nonlinearity(h)
    h = self.dropout(h)        # regularization
    h = self.conv2(h)

    # ================= Shortcut (residual) 처리 =================
    # in_channels != out_channels 인 경우 채널 mismatch 보정
    # U-Net down/up block에서 채널 수가 바뀔 때 필요
    if self.in_channels != self.out_channels:
        if self.use_conv_shortcut:
            # conv_shortcut: 일반 3×3 conv
            x = self.conv_shortcut(x)
        else:
            # nin_shortcut: 1×1 conv(Network-in-Network)
            x = self.nin_shortcut(x)

    # ================= 최종 residual 합 =================
    return x + h


In [ ]:
def forward(self, x, t=None, context=None):
    """
    x       : noisy latent feature map (B, C, H, W)
              - VAE encoder 결과 z에 noise가 섞인 상태
    t       : diffusion timestep (B,)
              - noise level을 나타내는 정수 timestep
    context : conditioning 정보 (optional)
              - LDM에서는 보통 text embedding (e.g. CLIP text encoder output)
              - spatially aligned된 경우 channel 방향으로 concat
    """

    # =========================================================
    # 1. Context conditioning (early fusion)
    # =========================================================
    if context is not None:
        # context가 latent와 spatially aligned되어 있다고 가정
        # channel 차원으로 concat하여 조건 정보를 입력에 직접 결합
        x = torch.cat((x, context), dim=1)

    # =========================================================
    # 2. Timestep embedding 생성
    # =========================================================
    if self.use_timestep:
        # diffusion model에서는 timestep conditioning이 필수
        assert t is not None

        # t → sinusoidal embedding (B, ch)
        temb = get_timestep_embedding(t, self.ch)

        # MLP 1층
        temb = self.temb.dense[0](temb)
        temb = nonlinearity(temb)

        # MLP 2층
        temb = self.temb.dense[1](temb)
    else:
        temb = None

    # =========================================================
    # 3. Downsampling path (Encoder)
    # =========================================================
    # skip connection에 사용할 feature들을 저장
    hs = [self.conv_in(x)]  # 입력 latent를 feature space로 사상

    for i_level in range(self.num_resolutions):
        for i_block in range(self.num_res_blocks):
            # ResNet block (앞에서 설명한 그 block)
            h = self.down[i_level].block[i_block](hs[-1], temb)

            # 해당 resolution에서 attention을 쓰는 경우
            if len(self.down[i_level].attn) > 0:
                h = self.down[i_level].attn[i_block](h)

            # skip connection 저장
            hs.append(h)

        # 마지막 resolution이 아니면 spatial downsampling
        if i_level != self.num_resolutions - 1:
            hs.append(self.down[i_level].downsample(hs[-1]))

    # =========================================================
    # 4. Middle block (Bottleneck)
    # =========================================================
    h = hs[-1]

    # ResNet block
    h = self.mid.block_1(h, temb)

    # Self-Attention (global context 통합)
    h = self.mid.attn_1(h)

    # ResNet block
    h = self.mid.block_2(h, temb)

    # =========================================================
    # 5. Upsampling path (Decoder)
    # =========================================================
    for i_level in reversed(range(self.num_resolutions)):
        for i_block in range(self.num_res_blocks + 1):
            # skip connection과 concat (U-Net 핵심)
            h = self.up[i_level].block[i_block](
                torch.cat([h, hs.pop()], dim=1),
                temb
            )

            # attention이 있는 경우 적용
            if len(self.up[i_level].attn) > 0:
                h = self.up[i_level].attn[i_block](h)

        # 마지막 level이 아니면 upsampling
        if i_level != 0:
            h = self.up[i_level].upsample(h)

    # =========================================================
    # 6. Output projection
    # =========================================================
    h = self.norm_out(h)
    h = nonlinearity(h)

    # 최종 출력: noise prediction (ε̂ or v̂)
    h = self.conv_out(h)

    return h
